<div align=center><h1>Individual Project Nº1</h1>
<h4>Salomón Orozco Jaramillo</h4>
<h4>ETL (Extract, Transform, Load)</h4>
</div>

## Importing Libs

In [ ]:
# Importing necessary libraries
import ast  # For working with Python literal syntax
import json  # For working with JSON files
import pandas as pd  # For data manipulation
import nltk  # Natural Language Toolkit
import re # Regular Expression library
import nltk # Use the nltk library with vader_lexicon and sentiment to transform reviews into sentiment scores
nltk.download('vader_lexicon') # Download  VADER lexicon from NLTK
from nltk.sentiment.vader import SentimentIntensityAnalyzer # Initiate  the sentiment analyzer


## australian_users_items.json etl

In [ ]:
# Creating a list for the items
review = []
# Opening json
with open('datasets/australian_users_items.json', 'r', encoding='utf-8') as usersItemJson:
    for line in usersItemJson:
        # Append json items to the list
        review.append(ast.literal_eval(line))
# Close the json file explicitly
usersItemJson.close()
# Creating a dataframe
df_user_items = pd.DataFrame(review)
# Viewing the dataframe
df_user_items

### Desempaquetar la columna 'Items'

In [ ]:
# Unpacking the 'Items' column
data_user_temp1 = df_user_items.explode(['items'])
# Normalizing JSON data in the 'items' column
data_user_temp2 = pd.json_normalize(data_user_temp1['items']).set_index(data_user_temp1['items'].index)
# Concatenating normalized data with the original DataFrame
data_user_temp3 = pd.concat([data_user_temp2, data_user_temp1], axis=1)
# Updating the original DataFrame with the concatenated data
df_user_items = data_user_temp3

In [ ]:
# Remove the 'items' column from the DataFrame
df_user_items = df_user_items.drop('items', axis=1)

In [ ]:
# View DataFrame
df_user_items

In [ ]:
# Check the 'item_id' content data for a specific user ('76561198329548331')
df_user_items.loc[df_user_items['user_id'] == '76561198329548331']

#df_user_items['steam_id'].info()

In [ ]:
# Calculate the number of null values per column in df_user_items
null_per_col = df_user_items.isnull().sum()

#null_per_col

In [ ]:
# Remove columns that are not needed
df_user_items = df_user_items.drop('user_url', axis=1)
df_user_items = df_user_items.drop('items_count', axis=1)

In [ ]:
# View DataFrame

df_user_items

In [ ]:
# Display information about df_user_items
df_user_items.info()

In [ ]:
# Find NaN values and drop corresponding rows for specific columns
df_user_items = df_user_items.dropna(subset=['item_id'])
df_user_items = df_user_items.dropna(subset=['item_name'])
df_user_items = df_user_items.dropna(subset=['playtime_forever'])
df_user_items = df_user_items.dropna(subset=['playtime_2weeks'])
df_user_items = df_user_items.dropna(subset=['steam_id'])
df_user_items = df_user_items.dropna(subset=['user_id'])

In [ ]:
# Dictionary mapping old column names to new column names
column_mapping = {
    'user_id': 'User_Id',
    'item_id': 'Item_Id',
    'playtime_forever': 'Playtime_Forever',
    'item_name': 'Item_Name',
    'steam_id': 'Steam_Id',
    'playtime_2weeks': 'Playtime_2Weeks'
}

# Rename columns using the dictionary
df_user_items = df_user_items.rename(columns=column_mapping)

In [ ]:
# View Dataframe
df_user_items

In [ ]:
# Get DataFrame info
df_user_items.info()
duplicates = df_user_items.duplicated()
print(duplicates.sum())

### Export Data Files

In [ ]:
# Save the DataFrame to a CSV file
df_user_items.to_csv('CSV/user_items.csv', index=False)


In [ ]:
# Save the DataFrame to a compressed Parquet file
df_user_items.to_parquet('GZIP/user_items.gzip', compression='gzip')

## output_steam_games.json etl

In [ ]:
# List to store JSON objects
games = []
# Read 'output_steam_games.json' line by line and load as JSON
with open('datasets/output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        games.append(json.loads(line))
# Create a DataFrame from the list of JSON objects
df_steam_games = pd.DataFrame(games)

In [ ]:
# View DataFrame
df_steam_games

In [ ]:
# Dictionary mapping old column names to new column names
column_mapping = {
    'publisher': 'Publisher',
    'genres': 'Genres',
    'app_name': 'App_Name',
    'title': 'Title',
    'url': 'Url',
    'release_date': 'Release_Date',
    'tags': 'Tags',
    'reviews_url': 'Reviews_Url',
    'specs': 'Specs',
    'price': 'Price',
    'early_access': 'Early_Access',
    'id': 'Id',
    'developer': 'Developer'
}
# Rename columns using the dictionary
df_steam_games = df_steam_games.rename(columns=column_mapping)

In [ ]:
# View new column names
df_steam_games.info()

In [ ]:
def normalize_col(col):
    """
    Normalize a developer name by removing non-alphabetic characters,
    converting to lowercase, and removing extra whitespaces.
    """
    if isinstance(col, str):  # Verify if it is a string
        cleaned_name = re.sub(r'[^a-zA-Z\s]', '', col)
        new_name = re.sub(r'\s+', ' ', cleaned_name.lower().strip())
        return new_name
    else:
        return col

# Apply the normalize_col function to the 'Publisher' column
df_steam_games['Publisher'] = df_steam_games['Publisher'].apply(normalize_col)


In [ ]:
# Remove the 'Url' column
df_steam_games.drop('Url', axis=1, inplace=True)

In [ ]:
df_steam_games

In [ ]:
# Function to convert float to string
def float_to_string(value):
    """
    Convert a float to string. If the value is not a float, return it unchanged.
    """
    if isinstance(value, float):
        return str(value)
    return value
# Apply the float_to_str function to the 'Publisher' column
df_steam_games['Publisher'] = df_steam_games['Publisher'].apply(float_to_string)


In [ ]:
# Capitalize the first letter of each word in the 'Publisher' column
df_steam_games['Publisher'] = df_steam_games['Publisher'].str.title()

In [ ]:
# Count Null values in 'Publisher'
nullPublisher = df_steam_games['Publisher'].isnull().sum()
print("Null values in 'Publisher':",nullPublisher)

# Count NaN values in 'Publisher'
nans_as_strings = (df_steam_games['Publisher'].str.lower() == 'nan').sum()
# Print the number of NaN values represented as strings in 'Publisher'
print("NaN values in 'Publisher':", nans_as_strings)
print('--------')
# Count Null values in 'Publisher'
nullPublisher = df_steam_games['Genres'].isnull().sum()
print("Null values in 'Genres':",nullPublisher)

In [ ]:
# Check Genres content
print(df_steam_games['Genres'])

In [ ]:
# Convert the values in the 'Genres' column to a comma-separated string
df_steam_games['Genres'] = df_steam_games['Genres'].astype(str).apply(lambda x: ', '.join(filter(None, x.split(','))))

In [ ]:
df_steam_games

In [ ]:
# Remove rows with NaN values in 'Genres'
df_steam_games.dropna(subset=['Genres'], inplace=True)

In [ ]:
df_steam_games['Genres']

In [ ]:
df_steam_games['Specs']

In [ ]:
# Count null values in 'Specs'
nullSpecs = df_steam_games['Specs'].isnull().sum()
# Print the number of null values in 'Specs'
print("Null values in 'Specs':", nullSpecs)


In [ ]:
# Remove rows with NaN values in 'Specs'
df_steam_games.dropna(subset=['Specs'], inplace=True)

In [ ]:
# Extract 'Specs' specifications
df_steam_games['Specs'] = df_steam_games['Specs'].apply(lambda x: ', '.join(filter(None, x)))


In [ ]:
df_steam_games

In [ ]:
# Count null values in 'title'
null_count = df_steam_games['Title'].isnull().sum()
# Print the number of null values in 'title'
print(f"Null values in 'Title': {null_count}")
# Count null app_name in 'App_name'
null_count = df_steam_games['App_Name'].isnull().sum()
# Print the number of null values in 'title'
print(f"Null values in 'App_Name': {null_count}")

In [ ]:
# Drop Title column
df_steam_games = df_steam_games.drop('Title', axis=1)

In [ ]:
# remove Na in App_Name
df_steam_games.dropna(subset=['App_Name'], inplace=True)

In [ ]:
df_steam_games

In [ ]:
# Normalize App_Name column names
df_steam_games['App_Name'] = df_steam_games['App_Name'].apply(normalize_col)

In [ ]:
# Capitalize App name column
df_steam_games['App_Name'] = df_steam_games['App_Name'].str.title()

In [ ]:
# Remove columns 'reviews_url' and 'tags'
remove_columns = ['Reviews_Url', 'Tags']
df_steam_games = df_steam_games.drop(columns=remove_columns)


In [ ]:
# View DataFrame
df_steam_games

In [ ]:
# Convert 'Release_Date' to datetime objects
df_steam_games['Release_Date'] = pd.to_datetime(df_steam_games['Release_Date'], dayfirst=True, errors='coerce')

In [ ]:
# Null columns in 'Release_Date'
null_release = df_steam_games['Release_Date'].isnull().sum()
print(null_release)

In [ ]:
# remove na from Release_Date
df_steam_games.dropna(subset = ['Release_Date'], inplace=True)

In [ ]:
df_steam_games

In [ ]:
# Add column 'Release_Year' with the year from 'Release_Date'
df_steam_games['Release_Year'] = pd.to_datetime(df_steam_games['Release_Date']).dt.year

In [ ]:
# View DataFrame
df_steam_games

In [ ]:
# apply float_to_string function to 'Genres' column
df_steam_games['Genres'] = df_steam_games['Genres'].apply(float_to_string)

In [ ]:
df_steam_games

In [ ]:
# Function to check if a value is a string and convert to 0
def string_to_zero(value):
    if isinstance(value, str):
        return 0
    else:
        return value

In [ ]:
# Check na in 'Price' column
df_steam_games['Price'].isna().sum()

In [ ]:
# Delete na rows in 'Price' column
df_steam_games.dropna(subset= ['Price'], inplace=True)

In [ ]:
# Apply the string_to_zero function in the free to play column
df_steam_games['Price'] = df_steam_games['Price'].apply(string_to_zero)

In [ ]:
# Check the number of non-numeric rows in the 'Price' column
non_numeric_count = pd.to_numeric(df_steam_games['Price'], errors='coerce').isnull().sum()
print(f"Number of rows with non-numeric values in 'Price': {non_numeric_count}")

In [ ]:
# Convert 'Price' column to float type
df_steam_games['Price'] = df_steam_games['Price'].astype(float)


In [ ]:
df_steam_games['Early_Access'].info()

In [ ]:
# Convert objet to bool
df_steam_games['Early_Access'] = df_steam_games['Early_Access'].astype(bool)

In [ ]:
# Count null values in 'Early_Access'
null_count = df_steam_games['Early_Access'].isnull().sum()
# Print the number of null values in 'Early_Access' column
print("\nNumber of null values in 'Early_Access' column:", null_count)

In [ ]:
# Check Developer names
df_steam_games

In [ ]:
# Normalize 'Developer' names
df_steam_games['Developer'] = df_steam_games['Developer'].apply(normalize_col)

In [ ]:
# Capitalize the first letter of each word in 'Developer' column
df_steam_games['Developer'] = df_steam_games['Developer'].str.title()

In [ ]:
# Count null values in 'Developer' column
null_count = df_steam_games['Developer'].isnull().sum()
# Print the number of null values in 'Developer' column
print("\nNumber of null values in 'Developer' column:", null_count)

In [ ]:
# Count null values in 'Developer' column
null_count = df_steam_games['Id'].isnull().sum()
# Print the number of null values in 'Developer' column
print("\nNumber of null values in 'Id' column:", null_count)

In [ ]:
# Drop rows with null values in 'Developer' column
df_steam_games.dropna(subset=['Developer'], inplace=True)

In [ ]:
# Drop rows with null values in 'Developer' column
df_steam_games.dropna(subset=['Id'], inplace=True)

In [ ]:
# Count null values in 'Developer' column
null_count = df_steam_games['Id'].isnull().sum()
# Print the number of null values in 'Developer' column
print("\nNumber of null values in 'id' column:", null_count)

In [ ]:
df_steam_games

In [ ]:
df_steam_games.columns

In [ ]:
# New columns order
new_order = ['Id','App_Name','Price','Release_Year','Release_Date','Genres','Early_Access','Specs','Developer', 'Publisher']
df_steam_games= df_steam_games[new_order]

In [ ]:
df_steam_games

In [ ]:
# Remove '[' and ']' characters as well as single quotes from the 'Genres' column
df_steam_games['Genres'] = df_steam_games['Genres'].str.replace(r"[\[\]']", '', regex=True)

In [ ]:
df_steam_games

In [ ]:
# Save the DataFrame to a Parquet file with gzip compression
df_steam_games.to_parquet('GZIP/df_steam_games.gzip', compression='gzip')
# Save the DataFrame to a CSV file without including the index column
df_steam_games.to_csv('CSV/steam_games.csv', index=False)

## australian_user_reviews.json etl

In [ ]:
# Creating a list for the items
review = []
# Opening json
with open('datasets/australian_user_reviews.json', 'r', encoding='utf-8') as usersRevJson:
    for line in usersRevJson:
        # Append json items to the list
        review.append(ast.literal_eval(line))
# Close the json file explicitly
usersItemJson.close()
# Creating a dataframe
df_user_rev = pd.DataFrame(review)
# Viewing the dataframe
df_user_rev

In [ ]:
# Explode data from the 'reviews' column and create new columns
allReviews = df_user_rev.explode('reviews') 
df_user_rev = pd.concat([allReviews.drop(['reviews'], axis=1), allReviews['reviews'].apply(pd.Series)], axis=1)
# View Dataframe
df_user_rev

In [ ]:
# Remove the first newly created column and the 'user_url' column
df_user_rev = df_user_rev.drop([0, 'user_url'], axis=1)
df_user_rev

In [ ]:
# Change the names of the columns
df_user_rev = df_user_rev.rename(columns={'user_id': 'User_Id', 'posted': 'Posted', 'item_id': 'Item_Id', 'helpful': 'Helpful', 'recommend': 'Recommend', 'review': 'Review'})
df_user_rev

In [ ]:
# Counting NaN values in the 'User_Id' column
nan_count = df_user_rev['User_Id'].isna().sum()
print("Number of NaN values in User_Id column:", nan_count)

In [ ]:
# Count empty values in the 'funny' column
empty_count = df_user_rev['funny'].value_counts()['']
# Calculate the total percentage of empty values relative to the total number of rows
total_rows = len(df_user_rev)
empty_percentage = (empty_count / total_rows) * 100
# Print the number of empty values in the 'funny' column
print("Number of empty values in the 'funny' column:", empty_count)
# Print the total percentage of empty values in the 'funny' column
print("Total percentage of empty values in the 'funny' column: {:.2f}%".format(empty_percentage))

In [ ]:
# Count empty values in the 'last_edited' column
empty_count_last_edited = (df_user_rev['last_edited'] == '').sum()
# Calculate the total percentage of empty values in the 'last_edited' column
total_rows = len(df_user_rev)
empty_percentage_last_edited = (empty_count_last_edited / total_rows) * 100
# Print the number of empty values in the 'last_edited' column
print("Number of empty values in the 'last_edited' column:", empty_count_last_edited)
# Print the total percentage of empty values in the 'last_edited' column
print("Total percentage of empty values in the 'last_edited' column: {:.2f}%".format(empty_percentage_last_edited))

In [ ]:
# Remove the 'funny' and 'last_edited' columns due to having more than 80% NaN values
df_user_rev = df_user_rev.drop(columns=['funny', 'last_edited'])


In [ ]:
# Count null 'posted'
null_in_Posted = df_user_rev['Posted'].isnull().sum()
print("Val Null 'posted':", null_in_Posted)

In [ ]:
# Drop rows with null values in the 'Posted' column
df_user_rev.dropna(subset=['Posted'], inplace=True)

In [ ]:
df_user_rev

In [ ]:
# Remove the text "Posted" from the dates in the 'Posted' column
df_user_rev['Posted'] = df_user_rev['Posted'].str.replace('Posted ', '')

In [ ]:
df_user_rev['Posted']

In [ ]:
# Format the data in the 'Posted' column to datetime, assigning NaT (Not a Time) to impossible formats
df_user_rev['Posted'] = pd.to_datetime(df_user_rev['Posted'], dayfirst=True, errors='coerce')


In [ ]:
df_user_rev

In [ ]:
# Remove rows with NaT (Not a Time) values in the 'Posted' column
df_user_rev.dropna(subset=['Posted'], inplace=True)

In [ ]:
# Check if the format is correct (YYYY-MM-DD)
df_user_rev['Posted'] = pd.to_datetime(df_user_rev['Posted']).dt.strftime('%Y-%m-%d')

In [ ]:
df_user_rev

In [ ]:
# Add a new column 'Year' containing the year extracted from the 'Posted' column
df_user_rev['Year'] = pd.to_datetime(df_user_rev['Posted']).dt.year

In [ ]:
# Display the count of null values in each column of the DataFrame
null_values = df_user_rev.isnull().sum()
print(null_values)
df_user_rev

In [ ]:
# Remove the 'Helpful' column from the DataFrame
df_user_rev = df_user_rev.drop(columns=['Helpful'])

In [ ]:
df_user_rev

In [ ]:
# Initialize the sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Function to assign sentiment scores according to the scale
def sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment_score = sentiment_analyzer.polarity_scores(text)
        sentiment_score = sentiment_score['compound']
        if sentiment_score >= -0.05:
            return 2  # Good score
        elif sentiment_score <= -0.05:
            return 0  # Bad score
        else:
            return 1  # Neutral score
    else:
        return 1  # Return neutral for non-string values

# Convert the 'Review' column to string
df_user_rev['Review'] = df_user_rev['Review'].astype(str)
# Apply the function get_sentiment_score to the 'Review' column
df_user_rev['Sentiment_Score'] = df_user_rev['Review'].apply(sentiment_score)
# View the DataFrame
df_user_rev


In [ ]:
# Remove the 'Review' column from the DataFrame
df_user_rev = df_user_rev.drop(columns=['Review'])

In [ ]:
df_user_rev

In [ ]:
# Save the DataFrame to a CSV file named 'user_rev.csv' without including the index
df_user_rev.to_csv('CSV/user_rev.csv', index=False)
# Also will save a PARQUET file too
df_user_rev.to_parquet('GZIP/df_user_revs.gzip', compression='gzip')


## Year Playtimes new Dataframe creation

In [ ]:
# Import the first round of cleaned datasets to create new datasets
df_steam_games = pd.read_parquet('GZIP/df_steam_games.gzip')
user_items = pd.read_parquet('GZIP/user_items.gzip')

In [ ]:

# Step 1: Group games by release year and genre, and calculate the total playtime per genre in each year
grouped_data = user_items.merge(df_steam_games[['Id', 'Release_Year', 'Genres']], 
                                 left_on='Item_Id', right_on='Id')\
                          .groupby(['Release_Year', 'Genres'])['Playtime_Forever']\
                          .sum()\
                          .reset_index()

# Step 2: Create a new DataFrame with columns 'Release_Year', 'Genres', and 'Total_Playtime'
hours_per_year = grouped_data.rename(columns={'Playtime_Forever': 'Total_Playtime'})

# Display the new dataset
hours_per_year

In [ ]:
# Save the hours_per_year in the CSV and GZIP folders
df_user_rev.to_csv('CSV/hours_per_year.csv', index=False)
# Also will save a PARQUET file too
df_user_rev.to_parquet('GZIP/dhours_per_year.gzip', compression='gzip')